# Ollama + OpenAI + Python

## 1. Määrake mudeli nimi

Kui laadisite teise mudeli kui "phi3:mini", muutke allolevas lahtris väärtust.
See muutuja kasutatakse kogu töövihiku koodis.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Seadista Open AI klient

Tavaliselt kasutatakse Open AI klienti OpenAI.com-i või Azure OpenAI-ga suurte keelemudelitega suhtlemiseks.
Siiski saab seda kasutada ka Ollamaga, kuna Ollama pakub OpenAI-ühilduvat lõpp-punkti aadressil "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Genereeri vestluse vastus

Nüüd saame OpenAI SDK-d kasutada, et genereerida vastus vestlusele. See päring peaks genereerima haiku kassidest:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Promptide inseneritöö

Esimene keelemudelile saadetav sõnum nimetatakse "süsteemisõnumiks" või "süsteemipromptiks" ja see määrab mudeli üldised juhised.  
Sa võid esitada oma kohandatud süsteemiprompti, et juhendada keelemudelit väljundi genereerimisel teisel viisil.  
Muuda allolevat `SYSTEM_MESSAGE`-i nii, et see vastaks nagu sinu lemmikkuulus filmi- või telesaate tegelane, või saa inspiratsiooni teiste süsteemipromptide jaoks lehelt [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Kui oled süsteemisõnumi kohandanud, esita esimene kasutajaküsimus `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Few-shot näited

Teine viis keelemudeli juhendamiseks on anda "few shots", järjestikune näidisküsimuste/‑vastuste seeria, mis demonstreerib, kuidas mudel peaks vastama.

Allolev näide püüab panna keelemudeli käituma nagu õppeassistent, esitades mõned näited küsimustest ja vastustest, mida õppeassistent võiks anda, ning seejärel esitab mudelile küsimuse, mida üliõpilane võiks küsida.

Proovige seda esmalt, ja seejärel muutke `SYSTEM_MESSAGE`, `EXAMPLES`, ja `USER_MESSAGE` uue stsenaariumi jaoks.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Otsinguga rikastatud genereerimine

RAG (Retrieval Augmented Generation) on tehnika, mille abil panna keelemudel vastama konkreetse valdkonna küsimustele täpselt, hankides esmalt asjakohast teavet teadmisteallikast ja seejärel genereerides vastuse selle teabe põhjal.

Meil on olemas kohalik CSV-fail, mis sisaldab andmeid hübriidautode kohta. Allolev kood loeb CSV-faili, otsib vastavusi kasutaja küsimusele ja genereerib seejärel leitud teabe põhjal vastuse. Pane tähele, et see võtab kauem aega kui eelmised näited, kuna mudelile saadetakse rohkem andmeid. Kui märkad, et vastus ei põhine siiski andmetel, võid proovida süsteemiinsenerimist või katsetada teisi mudeleid. Üldiselt on RAG tõhusam kas suuremate mudelitega või peenhäälestatud versioonidega SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
Vastutusest loobumine:
See dokument on tõlgitud tehisintellektil põhineva tõlketeenuse Co-op Translator (https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palun arvestage, et automatiseeritud tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument tema emakeeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitatakse kasutada professionaalset inimtõlget. Me ei vastuta käesoleva tõlke kasutamisest tulenevate arusaamatuste või väärtõlgenduste eest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
